# Data Retrieval

In [85]:
# Import dependancies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import csv
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
import matplotlib.pyplot as plt

### Get Vaccine Data from CDC

In [86]:
# Get Vaccinations Table from CDC website
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit("https://covid.cdc.gov/covid-data-tracker/#vaccinations")
vaccine_html = browser.html
browser.click_link_by_id("vaccinations-table-toggle")
cdc_html = browser.html
cdc_parsed = bs(cdc_html, "html.parser")
table = cdc_parsed.select("table")
browser.quit()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\coled\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [87]:
# Read the CDC Vaccine HTML Table
vaccines_df = pd.read_html(str(table))[0]

In [88]:
# Change "New York State" to "New York" for data merging
vaccines_df.loc[vaccines_df["State/Territory/Federal Entity"] == "New York State", "State/Territory/Federal Entity"] = "New York"

In [89]:
# Rename "State/Territory/Federal Entity" column to "state" for data merging
vaccines_df = vaccines_df.rename(columns={"State/Territory/Federal Entity": "state"})

In [90]:
# Overwrite Latest Vaccine CSV
vaccines_df.to_csv("../data/cdc-vaccine-data.csv")

### NYT / Rearc COVID Data

In [91]:
# Read Daily COVID-19 CSV from AWS S3 Bucket - Rearc / NY Times Data 
us_covid_cases_data = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-states/us-states.csv", stream=True)

In [92]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/us-covid-data.csv",  "wb") as file:
    file.write(us_covid_cases_data.content)

In [93]:
# Read US COVID Data CSV for data merging and cleaning 
us_covid_cases_df = pd.read_csv("../data/us-covid-data.csv")

In [94]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20029,2021-03-01,Virginia,51,577174,8783
20030,2021-03-01,Washington,53,343723,5038
20031,2021-03-01,West Virginia,54,132048,2300
20032,2021-03-01,Wisconsin,55,617700,7019


# State Overview Data

In [95]:
# Get yesterday's date, which is the latest data 
today = dt.date.today()
yesterday = today - dt.timedelta(days = 1)
latest_data = yesterday.strftime("%Y-%m-%d")

In [96]:
# Get latest total cases by state for latest date
us_total_cases_to_date_df = us_covid_cases_df.groupby(["date", "state"]).sum()
us_total_cases_to_date_df.reset_index(inplace=True)
us_total_cases_to_date_df = us_total_cases_to_date_df.loc[us_total_cases_to_date_df["date"] == latest_data]

In [97]:
# Read State Lat-Lon CSV and rename columns for easier cleaning and merging
state_latlons_df = pd.read_csv("../data/statelatlong.csv")
state_latlons_df.rename(columns={"State": "abbr", "Latitude": "lat", "Longitude": "lon", "City": "state"}, inplace=True)

In [98]:
# Read State Population CSV for cleaning and merging
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")

In [99]:
# Merge, rename, and focus on defined columns
state_overview_master_df = pd.merge(state_pop_df, state_latlons_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["state", "abbr", "lat", "lon", "population"]]

In [100]:
# Merge with us_total_cases_to_date_df and focus on defined columns
state_overview_master_df = pd.merge(state_overview_master_df, us_total_cases_to_date_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths"]]

In [101]:
# Merge with Vaccination data, rename columns, focus data columns
state_overview_master_df = pd.merge(state_overview_master_df, vaccines_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df.rename(columns={"Total Doses Administered by State where Administered": "total_doses_administered", "Doses Administered per 100k by State where Administered": "doses_administered_per_100k"})
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths", "total_doses_administered", "doses_administered_per_100k"]]

In [102]:
# Add Empty Column for New Calculation
state_overview_master_df["percent_vaccinated"] = ""

In [103]:
# Calcuation for percent vaccinated by state
for index, row in state_overview_master_df.iterrows():
    population = row["population"]
    vaccinated = row["total_doses_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_overview_master_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [104]:
# Convert percent vaccinated into a float
state_overview_master_df = state_overview_master_df.astype({"percent_vaccinated": 'float64'})

In [105]:
state_overview_master_df.to_csv("../data/state-master-data.csv")

# County Overview Data

In [106]:
populations_county = pd.read_csv("../data/census-bureau-population-by-county.csv")
poverty_county = pd.read_csv("../data/poverty-and-median-household-income-data-by-us-county-2019.csv")
state_keys = pd.read_csv("../data/state-names-codes.csv")

In [107]:
poverty_county = poverty_county[["Postal Code", "county", "Poverty Estimate, All Ages", "Median Household Income"]]
poverty_county = poverty_county.rename(columns={"Postal Code": "code", "Poverty Estimate, All Ages": "poverty_est_all_ages", 
                                              "Median Household Income": "median_household_income"})
poverty_county["county"] = poverty_county["county"].str.rstrip()

In [108]:
state_keys = state_keys.rename(columns={"Code": "code", "State": "state"})
state_keys = state_keys[["state", "code"]]

In [109]:
poverty_county = pd.merge(poverty_county, state_keys, how="inner", on="code")
poverty_county.drop(columns={"state"}, inplace=True)

In [110]:
populations_county = populations_county.merge(state_keys, how="inner", on="state")
populations_county = populations_county[["code", "county", "state", "2019_population_est"]]
populations_county["code"] = populations_county["code"].str.rstrip()
populations_county["county"] = populations_county["county"].str.rstrip()

In [111]:
merged_data = populations_county.merge(poverty_county, how='inner', left_on=["code", "county"], right_on=["code", "county"])

In [112]:
merged_data.dropna(how="any", inplace=True)

### Retrieve County Cases Data

In [113]:
county_cases = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-counties/us-counties.csv", stream=True)

In [114]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/county-covid-data.csv",  "wb") as file:
    file.write(county_cases.content)

In [115]:
# Read US COVID Data CSV for data merging and cleaning 
us_county_cases = pd.read_csv("../data/county-covid-data.csv")

In [116]:
us_county_cases = us_county_cases.groupby(["date", "state", "county"]).sum()
us_county_cases.reset_index(inplace=True)

In [117]:
latest_covid_cases = us_county_cases.loc[us_county_cases["date"] == latest_data]

In [118]:
county_cases = latest_covid_cases.merge(merged_data, how="left")

In [119]:
county_cases.dropna(how="any", inplace=True)

In [120]:
county_cases.to_csv("../data/county-master.csv")

In [140]:
county_cases

,date,state,county,fips,cases,deaths,code,2019_population_est,poverty_est_all_ages,median_household_income
0,2021-03-01,Alabama,Autauga,1001.0,6270,91.0,AL,55869.0,"6,723","58,233"
1,2021-03-01,Alabama,Baldwin,1003.0,19758,283.0,AL,223234.0,"22,360","59,871"
2,2021-03-01,Alabama,Barbour,1005.0,2116,51.0,AL,24686.0,"5,909","35,972"
3,2021-03-01,Alabama,Bibb,1007.0,2450,60.0,AL,22394.0,"4,101","47,918"
4,2021-03-01,Alabama,Blount,1009.0,6102,127.0,AL,57826.0,"9,324","52,902"
...,...,...,...,...,...,...,...,...,...,...
3261,2021-03-01,Wyoming,Sweetwater,56037.0,3772,34.0,WY,42343.0,"3,453","80,639"
3262,2021-03-01,Wyoming,Teton,56039.0,3357,9.0,WY,23464.0,"1,396","98,837"
3263,2021-03-01,Wyoming,Uinta,56041.0,2055,12.0,WY,20226.0,"1,699","70,756"
3264,2021-03-01,Wyoming,Washakie,56043.0,882,26.0,WY,7805.0,845,"55,122"


## Daily Cases

In [123]:
# Convert Dates to pd.Datetimes
us_covid_cases_df["date"] = pd.to_datetime(us_covid_cases_df["date"])

In [124]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20029,2021-03-01,Virginia,51,577174,8783
20030,2021-03-01,Washington,53,343723,5038
20031,2021-03-01,West Virginia,54,132048,2300
20032,2021-03-01,Wisconsin,55,617700,7019


In [125]:
# Create Daily New Cases Dataframe
daily_new_cases_df = us_covid_cases_df.groupby("date").sum()

In [126]:
daily_new_cases_df.reset_index(inplace=True)

In [127]:
# Add new Empty Columns for Daily New Cases and Deaths
daily_new_cases_df["daily_new_cases"] = ""
daily_new_cases_df["daily_new_deaths"] = ""

In [128]:
previous_day_cases = 0
previous_day_deaths = 0
    
for index, row in daily_new_cases_df.iterrows():
    
    # Cases
    new_cases = row["cases"] - previous_day_cases
    previous_day_cases = row["cases"]
    daily_new_cases_df.at[index, "daily_new_cases"] = new_cases
    
    # Deaths
    new_deaths = row["deaths"] - previous_day_deaths
    previous_day_deaths = row["deaths"]
    daily_new_cases_df.at[index, "daily_new_deaths"] = new_deaths

In [129]:
daily_new_cases_df

,date,fips,cases,deaths,daily_new_cases,daily_new_deaths
0,2020-01-21,53,1,0,1,0
1,2020-01-22,53,1,0,0,0
2,2020-01-23,53,1,0,0,0
3,2020-01-24,70,2,0,1,0
4,2020-01-25,76,3,0,1,0
...,...,...,...,...,...,...
401,2021-02-25,1762,28431065,508107,77804,2465
402,2021-02-26,1762,28509327,510283,78262,2176
403,2021-02-27,1762,28572021,511850,62694,1567
404,2021-02-28,1762,28622946,512979,50925,1129


In [130]:
daily_new_cases_df.to_csv("../data/state-daily-stats.csv")

### Set Up NYT Totals Master DF

In [132]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20029,2021-03-01,Virginia,51,577174,8783
20030,2021-03-01,Washington,53,343723,5038
20031,2021-03-01,West Virginia,54,132048,2300
20032,2021-03-01,Wisconsin,55,617700,7019


## Amazon RDS Updates

In [133]:
# Config Variables, and SQLalchemy
from config import endpoint, username, password
from sqlalchemy import create_engine

In [134]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [135]:
# 50 States & D.C. Data
state_overview_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)

In [136]:
engine.execute('ALTER TABLE master_table ADD PRIMARY KEY (state);')

In [137]:
# Daily Cases Data
daily_new_cases_df.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)

In [138]:
engine.execute('ALTER TABLE daily_new_cases ADD PRIMARY KEY (date);')

In [ ]:
# County Data
county_cases.to_sql("county_cases_master", index=False, if_exists='replace', con=connection)
engine.execute('ALTER TABLE county_cases_master ADD PRIMARY KEY (county);')

In [ ]:
# Raw NTY Data
us_covid_cases_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE nyt_table ADD PRIMARY KEY (cases);')